In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

In [2]:
data = open('data.txt', 'rb').read().decode(encoding='utf-8')

In [3]:
data

'pocgkr\ncgwpcfks\nkfsfkspwg\npfkswkfsococg\nwptgkfsoc\npwfkg\npowcer\ncoprerk\nwopofkkskf\nopwpwgcoc\npocrfksfks\ncgpogpoc\npwocgfkfsrere\nocwgrk\nwpgsfk\npwofsg\npocpocwgkfs\npocgwg\npwgockfsfksret\nocpwgfkskfsrere\ncopwgwpresk\npwterfkf\npocwggpfkfksrer\nwpofcgkrpwgfksocr\ncwpfksor\npwfksorc\npocgwkfkfser\npowctgcog\npowgcpo\npwfkt\npwokfkrg\ncopcogpgwfksr\npgsfkskfg\nwgptocopwg\npowcgre\nperkscwgtf\npcgworkfkfs\npopocgkr\npwtgcfkfksr\npocwkf\npgkfkf\ncwopg\npowcgkfkfr\nwpgfksk\npocwcpoere\npwkf\ncptgo\npocfkswgr\npowcfkfk\npwgfksfk\ngpwfks\ncgwpw\ncwopgtrekfksksf\npocsfkw\nocpwgfkf\npwgterkfsoc\npocgwrkfs\ntgcwpro\npocgkfskfr\npgcow\nopwg\nwpgcowpcfksre\nre\nptwgfks\npocrfkfk\npockr\npockfskfswgre\ntgpcowpgpwcog\ngwpfrkst\npfkfksg\npwogsfkg\npwkf\ngwpocoskfsfk\nwtpcockfkf\nwpgfkfkre\npococwtrfkfkg\npwocfksfksr\npwgerks\npoctfkfkrwg\npwfksocr\ntnpzc\ntnpcz\ntnpzc\ntnzpc\npmz\nxmbzd\npImz\nmzxp\npbz\npgz\npnzga\ndnzgdp\npndz\nnzpdgds\nmcddxp\npcfkdddaag\ndagpk\npc\ngkfadp\npfgds\nbcs

In [4]:
vocabulary = sorted(set(data))
len(vocabulary)

21

### Vectorize text

In [5]:
example_texts = ['abcdefg', 'xhz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'h', b'z']]>

In [6]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocabulary))

In [7]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[4, 5, 6, 7, 8, 9, 10], [21, 11, 22]]>

In [8]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [9]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'h', b'z']]>

In [10]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xhz'], dtype=object)

In [11]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### Create training samples + targets

In [12]:
all_ids = ids_from_chars(tf.strings.unicode_split(data, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1593,), dtype=int64, numpy=array([16, 15,  6, ...,  5,  2,  2])>

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [14]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

p
o
c
g
k
r


c
g
w


In [15]:
seq_length = 8
examples_per_epoch = len(data)//(seq_length+1)

In [16]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor([b'p' b'o' b'c' b'g' b'k' b'r' b'\n' b'c' b'g'], shape=(9,), dtype=string)


In [17]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'pocgkr\ncg'
b'wpcfks\nkf'
b'sfkspwg\np'
b'fkswkfsoc'
b'ocg\nwptgk'


In [18]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [19]:
dataset = sequences.map(split_input_target)

In [20]:
for input_example, target_example in  dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'pocgkr\nc'
Target: b'ocgkr\ncg'


### Create training batches

In [22]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 8), (64, 8)), types: (tf.int64, tf.int64)>

### Build model

In [23]:
# Length of the vocabulary in chars
vocab_size = len(vocabulary)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [31]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else: 
            return x

In [32]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

### Try out model

In [34]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 8, 23) # (batch_size, sequence_length, vocab_size)


In [35]:
model.summary()

Model: "my_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  5888      
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense_1 (Dense)              multiple                  23575     
Total params: 3,967,767
Trainable params: 3,967,767
Non-trainable params: 0
_________________________________________________________________


In [36]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [37]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'bagc\npkf'

Next Char Predictions:
 b'fkImfebr'


### Train model

In [39]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [40]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 8, 23)  # (batch_size, sequence_length, vocab_size)
Mean loss:         3.1360192


In [41]:
tf.exp(mean_loss).numpy()

23.01208

In [42]:
model.compile(optimizer='adam', loss=loss)

In [43]:
epochs = 20

In [45]:
history = model.fit(dataset, epochs=epochs)

Epoch 1/20
2/2 [==============================] - 0s 59ms/step - loss: 3.1168
Epoch 2/20
2/2 [==============================] - 0s 57ms/step - loss: 3.0194
Epoch 3/20
2/2 [==============================] - 0s 56ms/step - loss: 2.8521
Epoch 4/20
2/2 [==============================] - 0s 62ms/step - loss: 2.7164
Epoch 5/20
2/2 [==============================] - 0s 55ms/step - loss: 2.6189
Epoch 6/20
2/2 [==============================] - 0s 56ms/step - loss: 2.5839
Epoch 7/20
2/2 [==============================] - 0s 62ms/step - loss: 2.5110
Epoch 8/20
2/2 [==============================] - 0s 59ms/step - loss: 2.4006
Epoch 9/20
2/2 [==============================] - 0s 57ms/step - loss: 2.3336
Epoch 10/20
2/2 [==============================] - 0s 58ms/step - loss: 2.2845
Epoch 11/20
2/2 [==============================] - 0s 55ms/step - loss: 2.2151
Epoch 12/20
2/2 [==============================] - 0s 54ms/step - loss: 2.1668
Epoch 13/20
2/2 [==============================] - 0s 55ms/st

### Generate prediction

In [46]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature=temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "" or "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices = skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())]) 
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits] 
        predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [47]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [53]:
states = None
next_char = tf.constant(['p'])
result = [next_char]

for n in range(100):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

pzdksc
pwfks
preepopwockoowocore
wpwotrkeroowowrewreerreereeeeooowoocwoee
rkwkotretrrereekerrkreereow 

________________________________________________________________________________
